In [1]:
import ee
import pandas as pd

In [2]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [3]:
def country_check(country):
    location = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0").filter(ee.Filter.eq("ADM0_NAME", str(country)))
    if location.size().getInfo() > 0:
        Area = location.geometry().area()
        AreaSqKm = ee.Number(Area).divide(1e6).round().getInfo()
        if AreaSqKm < 9e6:
            return str("working")
        else:
            return str("size error")
    else:
        return str("name error")

In [4]:
df = pd.read_csv("country_time_index.csv")
countries_with_doubles = df["Area"].tolist()
seen = set()
countries = []
for item in countries_with_doubles:
    if item not in seen:
        seen.add(item)
        countries.append(item)

In [5]:
check_result = []
country_index = []
i = 0
while i < len(countries):
    result = country_check(country = countries[i])
    check_result.append(result)
    country_index.append(countries[i])
    i += 1

In [6]:
df_country_check = pd.DataFrame()
df_country_check["Area"] = country_index
df_country_check["ee check"] = check_result
df_country_check.to_csv("ee_country_check.csv")

In [8]:
ee_collection = df_country_check[df_country_check["ee check"].str.contains("working") == True
].reset_index().drop(columns=["index", "ee check"])

ee_collection.to_csv("ee_collection_dataset.csv")